In [24]:
import numpy as np
import os
import sys
import pdb
sys.path.append("..")

In [28]:
ground_truth_dict = {}
with open('test_images/train.txt','r') as annotation_file:
    for line in annotation_file:
        split = line.split(' ')
        name = split[0].split('.')[0]
        ground_truth = np.zeros((640, 480))
        for box in split[1:-1]:
            xmin, ymin, xmax, ymax = [int(pix) for pix in box.split(',')[:4]]
            ground_truth[xmin:xmax, ymin:ymax] = 1
        ground_truth_dict[name] = ground_truth


In [29]:
dir = 'faster_rcnn_results/'
for filename in os.listdir(dir):
    if filename.endswith(".npy"): 
        output_dict = np.load(dir + filename).item()
        detection_grid = np.zeros((640, 480))
        name = filename.split('.')[0]
        for i in range(len(output_dict['detection_boxes'])):
            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][i]
            xmin = int(xmin * 640)
            xmax = int(xmax * 640)
            ymin = int(ymin * 480)
            ymax = int(ymax * 480)
            score = output_dict['detection_scores'][i]
            box_grid = detection_grid[xmin:xmax, ymin:ymax]
            box_grid[box_grid < score] = score
            detection_grid[xmin:xmax, ymin:ymax] = box_grid
        # calculate IOU
        if(name in ground_truth_dict):           
            intersection = detection_grid[ground_truth_dict[name] == 1].sum()
            union =  ground_truth_dict[name].sum() + detection_grid.sum() - intersection
            iou = intersection / union
            print(name, iou)
        

set00_V013_107 0.28976567957440924
set00_V011_95 0.11638881577274676
set00_V010_139 0.349974794756047
set00_V012_148 0.42078463071352473
set00_V001_17 0.0
set00_V009_19 0.2634582600268027
set00_V006_113 0.5726527183079078
set00_V010_124 0.40832328136958407
set00_V012_0 0.0
set00_V000_158 0.06098266183202626
set00_V010_89 0.0
set00_V012_39 0.6973987022104363
set00_V008_26 0.19760320857604233
set00_V014_68 0.3532090637406233
set00_V004_147 0.03995851252153489
set00_V010_155 0.3957805531652735
set00_V011_9 0.2222230073692978
set00_V001_141 0.0
set00_V007_47 0.0
set00_V010_63 0.6860913893284102
set00_V007_124 0.1997045315784534
set00_V014_32 0.3609130641636055
set00_V013_164 0.31713036013781354
set00_V013_156 0.4210388406245893
set00_V004_125 0.0
set00_V013_24 0.0
set00_V000_180 0.0
set00_V000_118 0.0
set00_V001_6 0.0
set00_V013_158 0.47577917992137403
set00_V007_156 0.22100798394000845
set00_V007_73 0.6336692448044446
set00_V001_157 0.0
set00_V011_33 0.0
set00_V008_111 0.0
set00_V012_135 